In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import pandas as pd

# Import the CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Hardcode the username and password for the "aacuser" account
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', '32686')

# Retrieve all data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the '_id' column if present
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

# Add the Grazioso Salvare logo
image_filename = 'Grazioso_Salvare_Logo.png'  # Make sure the file is in the same directory
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    # Add the logo and unique identifier
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px'}),
                       href='https://www.snhu.edu')),
    html.Center(html.B(html.H1('CS-340 Dashboard - Zainab Lowe'))),
    html.Hr(),
    
    # Add filtering options
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            ],
            value='all',
            labelStyle={'display': 'inline-block'}
        ),
    ], style={'textAlign': 'center'}),
    
    html.Hr(),

    # Data table to display filtered data
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    
    html.Br(),
    html.Hr(),

    # Dashboard layout for charts
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update the data table based on the filter type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Define the query conditions based on the filter type
    if filter_type == 'water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        # Reset to retrieve all documents if 'all' is selected
        query = {}

    # Retrieve data from MongoDB using the defined query
    filtered_data = pd.DataFrame.from_records(shelter.read(query))

    # Drop the '_id' column if it exists
    if '_id' in filtered_data.columns:
        filtered_data.drop(columns=['_id'], inplace=True)

    # Return the filtered data
    return filtered_data.to_dict('records')

# Update the chart based on the filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    # Check if data is available
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available to display.")]
    
    # Convert data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create a pie chart
    fig = px.pie(dff, names='breed', title='Distribution of Breeds')
    
    # Return the chart
    return [dcc.Graph(figure=fig)]

# Update the map based on the selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available to display on the map.")]

    # Convert the data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Set the default row index
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    # Check if the DataFrame has the latitude and longitude columns
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return [html.P("Location coordinates are not available.")]

    # Get the latitude and longitude
    lat = dff.iloc[row]['location_lat']
    lon = dff.iloc[row]['location_long']

    # Create the map centered around the selected location
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]

# Run the app
app.run_server(debug=True)


Connection to MongoDB successful.
Dash app running on http://127.0.0.1:30980/
